In [ ]:
%pip install -r requirements.txt
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from phys_ml.util import slurm_generate
from phys_ml.trainer import TrainerModes, VertexTrainer
from phys_ml.load_data.vertex import AutoEncoderVertexDataset

### Create slurm-script

In [ ]:
# generate slurm
trainer_kwargs = {
    'project_name': 'vertex', 
    'config_name': 'confmod_auto_encoder.json', 
    'subconfig_name': 'AUTO_ENCODER_VERTEX', 
    'config_kwargs': {
        'path_train': '/gpfs/data/fs71925/shepp123/frgs',
        'subset': 10, 
        'subset_shuffle': True, 
        'subset_seed': 42,
    },
}
slurm_generate.create(project_name='vertex', script_name='vsc_test', 
                      pyenv_dir='/gpfs/data/fs71925/shepp123/anaconda3',
                      trainer='VertexTrainer', trainer_kwargs=trainer_kwargs)


In [ ]:
# create slurm- and train-script from config-file, using the default slurm-config name
!python phys_ml/util/slurm_generate.py configs/confmod_auto_encoder.json

In [ ]:
# create slurm- and train-script from config-file, specifying the name of the subsection containing the slurm-configs
!python phys_ml/util/slurm_generate.py configs/confmod_auto_encoder.json SLURM_CONFIG_24X6

# Demo: Vertex Autoencoder

### Load data for inspection

In [ ]:
vertex = AutoEncoderVertexDataset.load_from_file(path='../frgs/tp0.020000_mu0.080000.h5')
print(vertex.shape)

(576, 576, 576)


### Training

In [ ]:
trainer = VertexTrainer(project_name='vertex', config_name='confmod_auto_encoder.json', 
                        subconfig_name='AUTO_ENCODER_VERTEX', 
                        config_kwargs={'path_train': '/gpfs/data/fs71925/shepp123/frgs',
                                       'sample_count_per_vertex': 2000, 
                                       'subset': 10})

In [ ]:
trainer.train(train_mode=TrainerModes.JUPYTERGPU)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir saves

### Predict

In [ ]:
# run a prediction, result will be stored in the saves-folder
# works with a trained or untrained VertexTrainer
pred = trainer.predict(new_data_path='../frgs/tp0.020000_mu0.080000.h5', 
                       save_path='save_AUTO_ENCODER_VERTEX_BS20_2024-11-25')
pred.shape

In [ ]:
# load already existing prediction
pred = trainer.load_prediction(save_path='save_AUTO_ENCODER_VERTEX_BS20_2024-11-25/version_0')
pred.shape